In [1]:
!pip -q install -U "transformers>=4.41" "datasets>=2.19" stanza==1.5.0 conllu==6.0.0 seqeval==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 15.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 89.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━

In [2]:
import sys
sys.path.insert(0, "/kaggle/working/libs")

# purge any already-imported copies (including submodules)
for name in list(sys.modules.keys()):
    if name == "transformers" or name.startswith("transformers."):
        del sys.modules[name]
    if name == "tokenizers" or name.startswith("tokenizers."):
        del sys.modules[name]
    if name == "huggingface_hub" or name.startswith("huggingface_hub."):
        del sys.modules[name]

import transformers, tokenizers, huggingface_hub
print("transformers:", transformers.__version__, transformers.__file__)
print("tokenizers:", tokenizers.__version__, tokenizers.__file__)
print("huggingface_hub:", huggingface_hub.__version__, huggingface_hub.__file__)

transformers: 4.57.1 /usr/local/lib/python3.11/dist-packages/transformers/__init__.py
tokenizers: 0.22.1 /usr/local/lib/python3.11/dist-packages/tokenizers/__init__.py
huggingface_hub: 0.36.0 /usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py


In [3]:
%%writefile setup.py
import os
import shutil

def safe_makedirs(path: str):
    """Safely create directory path (handle existing files)."""
    path = os.path.normpath(path)
    if not path or path == '.':
        return
    if os.path.exists(path):
        if os.path.isdir(path):
            return
        else:
            os.remove(path)
    parent = os.path.dirname(path)
    if parent and parent != path:
        if os.path.exists(parent):
            if not os.path.isdir(parent):
                os.remove(parent)
                safe_makedirs(parent)
        else:
            safe_makedirs(parent)
    os.makedirs(path, exist_ok=True)

def main():
    print("Setting up (offline mode)…")

    # ✅ Create unified writable data directory
    data_dir = "/kaggle/working/data/ud"
    safe_makedirs(data_dir)

    # ✅ Define local dataset sources (read-only)
    en_source = "/kaggle/input/english-data"
    bn_source = "/kaggle/input/bangla-dataaaaaaaa"

    # ✅ Define expected files and their locations
    files = {
        "en_ewt-ud-train.conllu": os.path.join(en_source, "en_ewt-ud-train.conllu"),
        "en_ewt-ud-dev.conllu": os.path.join(en_source, "en_ewt-ud-dev.conllu"),
        "bn_bru-ud-test.conllu": os.path.join(bn_source, "bn_bru-ud-test.conllu"),
    }

    # ✅ Copy each file into /kaggle/working/data/ud/
    for fname, src in files.items():
        dst = os.path.join(data_dir, fname)
        if os.path.exists(dst):
            print(f"✔ {fname} already in place")
        elif os.path.exists(src):
            shutil.copy(src, dst)
            print(f"📂 Copied {fname} → {dst}")
        else:
            print(f"⚠ Missing source file: {src}")

    print("✅ Offline setup complete!")
    print(f"   All writable data is now in: {data_dir}")

if __name__ == "__main__":
    main()


Writing setup.py


In [4]:
!python setup.py


Setting up (offline mode)…
📂 Copied en_ewt-ud-train.conllu → /kaggle/working/data/ud/en_ewt-ud-train.conllu
📂 Copied en_ewt-ud-dev.conllu → /kaggle/working/data/ud/en_ewt-ud-dev.conllu
📂 Copied bn_bru-ud-test.conllu → /kaggle/working/data/ud/bn_bru-ud-test.conllu
✅ Offline setup complete!
   All writable data is now in: /kaggle/working/data/ud


In [5]:
%%writefile postaging.py
import os
# Quiet noisy libs BEFORE importing HF/Torch
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
import torch
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from conllu import parse_incr
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    Trainer, TrainingArguments, DataCollatorForTokenClassification, set_seed
)
from transformers.trainer_utils import EvalPrediction
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from itertools import product
from pathlib import Path

# Harmless warnings
warnings.filterwarnings("ignore", message="Some weights of")
warnings.filterwarnings("ignore", message="checkpoint")
warnings.filterwarnings("ignore", message="AdamW is deprecated")

# ===== Labels (UPOS) =====
LABEL_LIST = ['ADJ','ADP','ADV','AUX','CCONJ','DET','INTJ','NOUN','NUM','PART','PRON','PROPN','PUNCT','SCONJ','SYM','VERB','X']
LABEL2ID = {t:i for i,t in enumerate(LABEL_LIST)}
ID2LABEL = {i:t for t,i in LABEL2ID.items()}

# ===== Config =====
BASE_CFG = dict(
    model_name="xlm-roberta-base",
    max_len=128,
    train_bs=8,
    eval_bs=16,
    # tiny sweep
    sweep_lrs=[1e-5],     
    sweep_epochs=[5],    
    # multi-seed
    seeds=[42],          
)

def _set_all_seeds(seed:int):
    set_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ✅ FIXED cache path — always in /kaggle/working/cache/
def _load_ud(file_path:str) -> Dataset:
    """Load and cache UD data safely under /kaggle/working/cache/"""
    cache_dir = "/kaggle/working/cache"
    os.makedirs(cache_dir, exist_ok=True)
    cache = os.path.join(cache_dir, os.path.basename(file_path) + ".cache")

    if os.path.exists(cache):
        print(f"Loading cached dataset: {cache}")
        return Dataset.load_from_disk(cache)

    print(f"Parsing: {file_path}")
    rows = []
    with open(file_path, "r", encoding="utf-8") as f:
        for tokenlist in parse_incr(f):
            rows.append({
                "tokens": [t["form"] for t in tokenlist],
                "upos": [t["upos"] for t in tokenlist],
            })

    ds = Dataset.from_pandas(pd.DataFrame(rows))
    ds.save_to_disk(cache)
    print(f"Saved cache: {cache}")
    return ds

def _to_ids(examples):
    return {"upos":[[LABEL2ID.get(tag, LABEL2ID['X']) for tag in tags] for tags in examples["upos"]]}

def _tokenize_fn(tokenizer, max_len):
    def f(examples):
        tok = tokenizer(
            examples["tokens"],
            truncation=True, padding="max_length",
            max_length=max_len, is_split_into_words=True
        )
        labels = []
        for i, lab in enumerate(examples["upos"]):
            word_ids = tok.word_ids(batch_index=i)
            prev = None; ids = []
            for w in word_ids:
                if w is None:
                    ids.append(-100)
                elif w != prev:
                    ids.append(lab[w])
                else:
                    ids.append(-100)
                prev = w
            ids += [-100]*(max_len - len(ids))
            labels.append(ids[:max_len])
        tok["labels"] = labels
        return tok
    return f

def _flatten_preds_labels(pred_logits, dataset):
    preds = np.argmax(pred_logits, axis=2)
    y_true_ids, y_pred_ids = [], []
    for i in range(len(dataset)):
        labels = dataset[i]["labels"]
        mask = np.array(labels) != -100
        y_true_ids.extend(np.array(labels)[mask].tolist())
        y_pred_ids.extend(np.array(preds[i])[mask].tolist())
    return y_true_ids, y_pred_ids

def _metrics_from_ids(y_true_ids, y_pred_ids):
    y_true = [ID2LABEL[i] for i in y_true_ids]
    y_pred = [ID2LABEL[i] for i in y_pred_ids]
    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, labels=LABEL_LIST, average="macro", zero_division=0)
    weighted_f1 = f1_score(y_true, y_pred, labels=LABEL_LIST, average="weighted", zero_division=0)
    report = classification_report(y_true, y_pred, labels=LABEL_LIST, digits=4, zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=LABEL_LIST)  # counts
    return acc, macro_f1, weighted_f1, report, cm

def _plot_confusion(cm_norm, title, out_png):
    plt.figure(figsize=(12,10))
    sns.heatmap(cm_norm, annot=False, cmap="Blues", xticklabels=LABEL_LIST, yticklabels=LABEL_LIST)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

# compute_metrics for Trainer (so eval logs include accuracy each epoch)
def _compute_metrics(eval_pred: EvalPrediction):
    preds = np.argmax(eval_pred.predictions, axis=2)
    labels = eval_pred.label_ids
    mask = labels != -100
    total = mask.sum()
    correct = ((preds == labels) & mask).sum()
    acc = float(correct) / float(total) if total > 0 else 0.0
    return {"accuracy": acc}

def _plot_learning_curve_from_logs(log_history, out_png, title="Learning Curve (EN-dev)"):
    epochs, eval_loss, eval_acc = [], [], []
    train_epochs, train_loss = [], []
    for rec in log_history:
        if "eval_loss" in rec:
            epochs.append(rec.get("epoch"))
            eval_loss.append(rec["eval_loss"])
            eval_acc.append(rec.get("eval_accuracy"))
        elif "loss" in rec and "epoch" in rec:
            train_epochs.append(rec["epoch"])
            train_loss.append(rec["loss"])
    plt.figure(figsize=(10,6))
    if train_epochs:
        plt.plot(train_epochs, train_loss, label="train_loss", marker="o")
    if epochs:
        plt.plot(epochs, eval_loss, label="eval_loss", marker="o")
    if epochs and any(a is not None for a in eval_acc):
        ax = plt.gca()
        ax2 = ax.twinx()
        ax2.plot(epochs, eval_acc, label="eval_accuracy", marker="s", linestyle="--", color="tab:green")
        ax2.set_ylabel("Eval Accuracy")
        ax2.legend(loc="lower right")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(title)
    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

def run_pos_tagging():
    print("\n" + "="*50 + "\nPOS TAGGING EVALUATION (multi-seed + sweep + learning curves)\n" + "="*50)
    out_root = Path("results_msweep"); out_root.mkdir(exist_ok=True, parents=True)

    # ===== 1) Load & prep data =====
    print("Loading datasets…")
    en_train = _load_ud("data/ud/en_ewt-ud-train.conllu").map(_to_ids, batched=True)
    en_dev   = _load_ud("data/ud/en_ewt-ud-dev.conllu").map(_to_ids, batched=True)
    bn_test  = _load_ud("data/ud/bn_bru-ud-test.conllu").map(_to_ids, batched=True)
    print(f"Train {len(en_train)}  Dev {len(en_dev)}  Test {len(bn_test)}")

    tokenizer = AutoTokenizer.from_pretrained(BASE_CFG["model_name"])
    tok_fn = _tokenize_fn(tokenizer, BASE_CFG["max_len"])

    print("Tokenizing…")
    en_train_tok = en_train.map(tok_fn, batched=True, remove_columns=en_train.column_names)
    en_dev_tok   = en_dev.map(  tok_fn, batched=True, remove_columns=en_dev.column_names)
    bn_test_tok  = bn_test.map( tok_fn, batched=True, remove_columns=bn_test.column_names)

    # ===== 2) Zero-shot baseline on BN-test (no selection) =====
    print("\n--- ZERO-SHOT (bn_test) ---")
    zs_model = AutoModelForTokenClassification.from_pretrained(
        BASE_CFG["model_name"], num_labels=len(LABEL_LIST),
        id2label=ID2LABEL, label2id=LABEL2ID, ignore_mismatched_sizes=True
    )
    zs_trainer = Trainer(
        model=zs_model,
        args=TrainingArguments(
            output_dir=str(out_root/"zero_shot"),
            per_device_eval_batch_size=BASE_CFG["eval_bs"],
            report_to="none",
            dataloader_num_workers=2,
            dataloader_pin_memory=True,
            fp16=torch.cuda.is_available(),
        ),
        data_collator=DataCollatorForTokenClassification(tokenizer)
    )
    zs_pred = zs_trainer.predict(bn_test_tok)
    zs_true_ids, zs_pred_ids = _flatten_preds_labels(zs_pred.predictions, bn_test_tok)
    zs_acc, zs_mf1, zs_wf1, zs_report, zs_cm = _metrics_from_ids(zs_true_ids, zs_pred_ids)
    (out_root/"zero_shot").mkdir(exist_ok=True, parents=True)
    with open(out_root/"zero_shot"/"bn_test_report.txt","w",encoding="utf-8") as f:
        f.write(f"acc={zs_acc:.6f}  macroF1={zs_mf1:.6f}  weightedF1={zs_wf1:.6f}\n\n{zs_report}")
    zs_cm_norm = zs_cm / (zs_cm.sum(axis=1, keepdims=True) + 1e-9)
    _plot_confusion(zs_cm_norm, "Zero-shot Confusion (BN test)", out_root/"zero_shot"/"bn_confusion.png")
    del zs_model; torch.cuda.empty_cache()

    # ===== 3) Multi-seed tiny sweep + learning curves =====
    runs = []
    aggregate_cm_counts = np.zeros((len(LABEL_LIST), len(LABEL_LIST)), dtype=np.int64)

    for seed in BASE_CFG["seeds"]:
        print(f"\n===== SEED {seed} =====")
        _set_all_seeds(seed)
        best = dict(macro_f1=-1, lr=None, epochs=None, trainer=None, model_dir=None)

        for lr, epochs in product(BASE_CFG["sweep_lrs"], BASE_CFG["sweep_epochs"]):
            print(f"  >> Try LR={lr}, Epochs={epochs}")
            _set_all_seeds(seed)
            model = AutoModelForTokenClassification.from_pretrained(
                BASE_CFG["model_name"], num_labels=len(LABEL_LIST),
                id2label=ID2LABEL, label2id=LABEL2ID, ignore_mismatched_sizes=True
            )
            args = TrainingArguments(
                output_dir=str(out_root/f"tmp_s{seed}_lr{lr}_ep{epochs}"),
                learning_rate=lr,
                per_device_train_batch_size=BASE_CFG["train_bs"],
                per_device_eval_batch_size=BASE_CFG["eval_bs"],
                num_train_epochs=epochs,
                eval_strategy="epoch",     
                save_strategy="no",
                logging_strategy="epoch",
                report_to="none",
                fp16=torch.cuda.is_available(),
                remove_unused_columns=True,
                dataloader_num_workers=2,
                dataloader_pin_memory=True,
                seed=seed,
            )
            trainer = Trainer(
                model=model, args=args,
                train_dataset=en_train_tok, eval_dataset=en_dev_tok,
                data_collator=DataCollatorForTokenClassification(tokenizer),
                compute_metrics=_compute_metrics
            )
            trainer.train()

            # Dev evaluation for selection (macro-F1)
            dev_pred = trainer.predict(en_dev_tok)
            y_true_ids, y_pred_ids = _flatten_preds_labels(dev_pred.predictions, en_dev_tok)
            dev_macro_f1 = f1_score([ID2LABEL[i] for i in y_true_ids],
                                    [ID2LABEL[i] for i in y_pred_ids],
                                    labels=LABEL_LIST, average="macro", zero_division=0)
            print(f"     dev macro-F1 = {dev_macro_f1:.4f}")

            if dev_macro_f1 > best["macro_f1"]:
                best.update(macro_f1=dev_macro_f1, lr=lr, epochs=epochs, trainer=trainer, model_dir=args.output_dir)
            else:
                # free GPU
                del trainer.model; del trainer; torch.cuda.empty_cache()

        # Use best trainer/model for this seed → Final EN-dev & BN-test
        seed_dir = out_root/f"seed_{seed}"; Path(seed_dir).mkdir(exist_ok=True, parents=True)
        print(f"Best for seed {seed}: LR={best['lr']}  Epochs={best['epochs']}  (dev macro-F1={best['macro_f1']:.4f})")

        # Save best model + tokenizer
        best_dir = Path(seed_dir) / "best_model"
        best["trainer"].save_model(str(best_dir))
        tokenizer.save_pretrained(str(best_dir))
        print(f"Saved best model for seed {seed} to: {best_dir}")

        # Learning curve from best trainer logs (EN-dev)
        log_hist = best["trainer"].state.log_history
        pd.DataFrame(log_hist).to_csv(Path(seed_dir)/"training_log_history.csv", index=False)
        _plot_learning_curve_from_logs(
            log_hist,
            Path(seed_dir)/"learning_curve.png",
            title=f"Learning Curve (EN-dev) — seed {seed}, lr={best['lr']}, ep={best['epochs']}"
        )

        # EN-dev final report
        dev_pred = best["trainer"].predict(en_dev_tok)
        dev_true_ids, dev_pred_ids = _flatten_preds_labels(dev_pred.predictions, en_dev_tok)
        dev_acc, dev_mf1, dev_wf1, dev_report, dev_cm = _metrics_from_ids(dev_true_ids, dev_pred_ids)
        with open(Path(seed_dir)/"en_dev_report.txt","w",encoding="utf-8") as f:
            f.write(f"acc={dev_acc:.6f}  macroF1={dev_mf1:.6f}  weightedF1={dev_wf1:.6f}\n\n{dev_report}")
        dev_cm_norm = dev_cm / (dev_cm.sum(axis=1, keepdims=True) + 1e-9)
        _plot_confusion(dev_cm_norm, f"EN-dev Confusion (seed {seed})", Path(seed_dir)/"en_confusion.png")

        # BN-test final report (held-out)
        test_pred = best["trainer"].predict(bn_test_tok)
        test_true_ids, test_pred_ids = _flatten_preds_labels(test_pred.predictions, bn_test_tok)
        test_acc, test_mf1, test_wf1, test_report, test_cm = _metrics_from_ids(test_true_ids, test_pred_ids)
        with open(Path(seed_dir)/"bn_test_report.txt","w",encoding="utf-8") as f:
            f.write(f"acc={test_acc:.6f}  macroF1={test_mf1:.6f}  weightedF1={test_wf1:.6f}\n\n{test_report}")
        test_cm_norm = test_cm / (test_cm.sum(axis=1, keepdims=True) + 1e-9)
        _plot_confusion(test_cm_norm, f"BN-test Confusion (seed {seed})", Path(seed_dir)/"bn_confusion.png")

        # Accumulate results
        runs.append(dict(seed=seed,
                         en=dict(acc=dev_acc, macro_f1=dev_mf1, weighted_f1=dev_wf1),
                         bn=dict(acc=test_acc, macro_f1=test_mf1, weighted_f1=test_wf1)))
        aggregate_cm_counts += test_cm

        # free GPU
        del best["trainer"].model; del best["trainer"]; torch.cuda.empty_cache()

    # ===== 4) Aggregate across seeds =====
    def _agg(metric):
        en_vals = [r["en"][metric] for r in runs]
        bn_vals = [r["bn"][metric] for r in runs]
        return (np.mean(en_vals), np.std(en_vals), np.mean(bn_vals), np.std(bn_vals))

    mean_en_acc, std_en_acc, mean_bn_acc, std_bn_acc = _agg("acc")
    mean_en_mf1, std_en_mf1, mean_bn_mf1, std_bn_mf1 = _agg("macro_f1")
    mean_en_wf1, std_en_wf1, mean_bn_wf1, std_bn_wf1 = _agg("weighted_f1")

    agg_cm_norm = aggregate_cm_counts / (aggregate_cm_counts.sum(axis=1, keepdims=True) + 1e-9)
    _plot_confusion(agg_cm_norm, "BN-test Confusion (Aggregate over seeds)", out_root/"bn_confusion_aggregate.png")

    summary = f"""FINAL SUMMARY over {len(runs)} seed(s)
EN-dev : acc={mean_en_acc:.4f}±{std_en_acc:.4f} | macroF1={mean_en_mf1:.4f}±{std_en_mf1:.4f} | weightedF1={mean_en_wf1:.4f}±{std_en_wf1:.4f}
BN-test: acc={mean_bn_acc:.4f}±{std_bn_acc:.4f} | macroF1={mean_bn_mf1:.4f}±{std_bn_mf1:.4f} | weightedF1={mean_bn_wf1:.4f}±{std_bn_wf1:.4f}

Zero-shot BN-test (single run): acc={zs_acc:.4f} | macroF1={zs_mf1:.4f} | weightedF1={zs_wf1:.4f}
"""
    print("\n" + summary)
    out_root.joinpath("summary.txt").write_text(summary, encoding="utf-8")

    return {
        "zero_shot_bn": dict(acc=zs_acc, macro_f1=zs_mf1, weighted_f1=zs_wf1),
        "runs": runs,
        "mean_std": {
            "en_dev": {
                "acc": (mean_en_acc, std_en_acc),
                "macro_f1": (mean_en_mf1, std_en_mf1),
                "weighted_f1": (mean_en_wf1, std_en_wf1),
            },
            "bn_test": {
                "acc": (mean_bn_acc, std_bn_acc),
                "macro_f1": (mean_bn_mf1, std_bn_mf1),
                "weighted_f1": (mean_bn_wf1, std_bn_wf1),
            },
        }
    }

if __name__ == "__main__":
    run_pos_tagging()


Writing postaging.py


In [6]:
from postaging import run_pos_tagging
results = run_pos_tagging()

E0000 00:00:1761776736.019000      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761776736.078407      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



POS TAGGING EVALUATION (multi-seed + sweep + learning curves)
Loading datasets…
Parsing: data/ud/en_ewt-ud-train.conllu


Saving the dataset (0/1 shards):   0%|          | 0/12544 [00:00<?, ? examples/s]

Saved cache: /kaggle/working/cache/en_ewt-ud-train.conllu.cache


Map:   0%|          | 0/12544 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.397900,0.121926,0.967934
2,0.090100,0.100012,0.974089
3,0.064100,0.094831,0.976441
4,0.049700,0.096202,0.976715
5,0.042500,0.095853,0.977107


     dev macro-F1 = 0.9670
Best for seed 42: LR=1e-05  Epochs=5  (dev macro-F1=0.9670)
Saved best model for seed 42 to: results_msweep/seed_42/best_model


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



FINAL SUMMARY over 1 seed(s)
EN-dev : acc=0.9771±0.0000 | macroF1=0.9670±0.0000 | weightedF1=0.9770±0.0000
BN-test: acc=0.8688±0.0000 | macroF1=0.6046±0.0000 | weightedF1=0.8701±0.0000

Zero-shot BN-test (single run): acc=0.1469 | macroF1=0.0203 | weightedF1=0.0721



In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch

MODEL_DIR = "/kaggle/working/results_msweep/seed_42/best_model"  # pick your seed’s folder

tok = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForTokenClassification.from_pretrained(MODEL_DIR)
device = 0 if torch.cuda.is_available() else -1

tagger = pipeline("token-classification", model=model, tokenizer=tok, device=device, aggregation_strategy="simple")

for s in ["আমি আজ বাজারে যাচ্ছি।", "বাংলা ভাষা খুব সুন্দর।"]:
    print("\nTEXT:", s)
    for item in tagger(s):
        print(f"{item['word']:>15s} -> {item['entity_group']}")


Device set to use cuda:0



TEXT: আমি আজ বাজারে যাচ্ছি।
            আমি -> PRON
             আজ -> ADV
          বাজার -> NOUN
              ে -> ADP
         যাচ্ছি -> VERB
              । -> PUNCT

TEXT: বাংলা ভাষা খুব সুন্দর।
          বাংলা -> ADJ
           ভাষা -> NOUN
            খুব -> ADV
         সুন্দর -> ADJ
              । -> PUNCT
